# Módulo 8 - Regressão Linear Múltipla


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### AED


#### Carga do Dados


Nesta aula, aprendemos a importância de trabalhar com nomes de colunas mais técnicos em um DataFrame. Isso facilita a referência e evita confusões com acentuação, maiúsculas e minúsculas. Aprendemos a renomear as colunas usando o atributo "columns" do DataFrame, atribuindo uma lista com os novos nomes das colunas desejadas. Também aprendemos sobre o conceito de Bucketing, que consiste em transformar variáveis numéricas em variáveis categóricas para análises específicas. Para isso, fizemos cópias do DataFrame original para trabalhar na análise exploratória e no Bucketing, deixando o DataFrame original intacto para o treinamento do modelo.


In [2]:
# carregar o dataset
df_colesterol = pd.read_csv("../datasets/dataset_colesterol.csv")
df_colesterol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [3]:
# remover coluna "Id"
df_colesterol.drop(columns=["Id"], axis=1, inplace=True)
df_colesterol.head(3)

,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,A,Não,Moderado,68.0,105.0,184.0,236.98
2,O,Não,Alto,25.0,64.8,180.0,161.79


In [4]:
# renomear colunas
df_colesterol.columns = [
    "grupo_saguineo",
    "fumante",
    "nivel_atividade_fisica",
    "idade",
    "peso",
    "altura",
    "colesterol",
]

In [5]:
# copiar DF para DF AED
df_colesterol_eda = df_colesterol.copy()
df_colesterol_eda.head(2)

,grupo_saguineo,fumante,nivel_atividade_fisica,idade,peso,altura,colesterol
0,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,A,Não,Moderado,68.0,105.0,184.0,236.98


In [6]:
# copiar DF para DF Bucketing
df_colesterol_bucketing = df_colesterol.copy()
df_colesterol_bucketing.tail(2)

,grupo_saguineo,fumante,nivel_atividade_fisica,idade,peso,altura,colesterol
998,AB,Sim,Baixo,61.0,91.2,161.0,284.53
999,A,Não,Baixo,45.0,82.4,163.0,237.45


#### Lidando com Valores Ausentes


Nesta aula, discutimos sobre a escolha adequada de medidas estatísticas para tratar valores ausentes em um DataFrame. Primeiramente, analisamos as medidas estatísticas das variáveis numéricas, como idade, peso e altura, e identificamos possíveis outliers. Em seguida, concluímos que a mediana seria uma escolha mais adequada para tratar valores ausentes, pois ela é menos sensível a outliers. Para as variáveis categóricas, utilizamos a moda, ou seja, o valor mais frequente da base. Demonstramos como realizar a imputação desses valores ausentes no DataFrame, coletando as medidas estatísticas e preenchendo os valores de uma vez só. Por fim, ressaltamos a importância de verificar o domínio dos tipos possíveis das variáveis categóricas e a necessidade de converter as variáveis numéricas para o tipo adequado.


In [7]:
# visualizar dados
df_colesterol_eda.head(7)

,grupo_saguineo,fumante,nivel_atividade_fisica,idade,peso,altura,colesterol
0,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,A,Não,Moderado,68.0,105.0,184.0,236.98
2,O,Não,Alto,25.0,64.8,180.0,161.79
3,A,Não,Alto,43.0,120.2,167.0,336.24
4,AB,Não,Baixo,79.0,88.5,175.0,226.23
5,B,Não,Baixo,68.0,66.8,170.0,185.31
6,A,Sim,Baixo,60.0,117.3,181.0,289.33


In [8]:
# visualizar valores únicos das colunas
print(f"Grupos Sanguíneos do paciente: {df_colesterol_eda['grupo_saguineo'].unique()}")
print(f"Paciente é Fumante?: {df_colesterol_eda['fumante'].unique()}")
print(
    f"Nivel de atividade física do paciente: {df_colesterol_eda['nivel_atividade_fisica'].unique()}"
)

Grupos Sanguíneos do paciente: ['B' 'A' 'O' 'AB' nan]
Paciente é Fumante?: ['Sim' 'Não' nan]
Nivel de atividade física do paciente: ['Baixo' 'Moderado' 'Alto' nan]


In [16]:
# detectar quantidade valores ausentes (nulos) por coluna
df_colesterol_eda.isna().sum()

grupo_saguineo            0
fumante                   0
nivel_atividade_fisica    0
idade                     0
peso                      0
altura                    0
colesterol                0
dtype: int64

In [10]:
# medidas estatísticas do DataFrame
df_colesterol_eda.describe()

,idade,peso,altura,colesterol
count,997.000000,997.000000,997.000000,1000.000000
mean,49.206620,76.496690,174.889669,199.453260
std,17.476704,21.439132,8.844703,50.625152
min,20.000000,15.400000,160.000000,36.260000
25%,34.000000,61.700000,167.000000,166.345000
50%,49.000000,75.500000,175.000000,199.455000
75%,65.000000,90.200000,183.000000,233.785000
max,79.000000,158.100000,190.000000,372.670000


In [11]:
# coletar medidas de variáveis categóricas
moda_grupo_sanguineo = df_colesterol_eda["grupo_saguineo"].mode()
moda_fumante = df_colesterol_eda["fumante"].mode()
moda_nivel_atividade_fisica = df_colesterol_eda["nivel_atividade_fisica"].mode()

In [12]:
# coletar medidade de variáveis numéricas
mediana_idade = df_colesterol_eda["idade"].median()
mediana_peso = df_colesterol_eda["peso"].median()
mediana_altura = df_colesterol_eda["altura"].median()

In [15]:
# imputar valores ausentes
df_colesterol_eda.fillna(
    value={
        "grupo_saguineo": moda_grupo_sanguineo[0],
        "fumante": moda_fumante[0],
        "nivel_atividade_fisica": moda_nivel_atividade_fisica[0],
        "idade": mediana_idade,
        "peso": mediana_peso,
        "altura": mediana_altura,
    },
    inplace=True,
)

#### Convertendo Variáveis Numéricas


Nesta aula, aprendemos a converter os tipos de dados em um DataFrame. Primeiro, tratamos os valores ausentes para evitar erros durante a conversão. Em seguida, utilizamos o método `type` para alterar o tipo das variáveis. No exemplo, convertemos a idade e a altura para inteiros. É importante realizar o tratamento dos valores ausentes antes da conversão, pois isso evita erros. Ao final da aula, verificamos que o DataFrame agora possui as colunas de idade e altura como inteiros, além de ter resolvido os valores ausentes.


#### Tratamento de Outliers


Nesta aula, vamos começar a análise dos dados que já foram preparados. Vamos focar na identificação de outliers, começando pela variável de peso. Utilizaremos um boxplot para visualizar os outliers e decidir se devemos removê-los. Em seguida, faremos o mesmo para a variável de altura e nível de colesterol. Verificamos que não há outliers na altura, mas há valores extremos no nível de colesterol, que decidimos manter no estudo. Em seguida, filtramos as pessoas com peso abaixo de 40 quilos, considerando que esses dados podem ter sido inseridos incorretamente. Removemos essas pessoas do DataFrame e reduzimos o público do estudo. Por fim, verificamos novamente os boxplots e confirmamos que não há outliers na idade, mas há outliers no peso. Decidimos remover as pessoas com peso muito baixo, pois não fazem sentido dentro do contexto do estudo.


#### Análise Grafica


Neste vídeo, continuamos nossa análise exploratória de dados, agora focando em cruzamentos entre variáveis categóricas e o nível de colesterol. Utilizamos o boxplot para visualizar a variação do nível de colesterol de acordo com diferentes categorias, como grupo sanguíneo, tabagismo e nível de atividade física. Observamos que o grupo sanguíneo pode influenciar o nível de colesterol, com os tipos O e B apresentando níveis menores em comparação aos tipos A e AB. Além disso, identificamos que fumantes tendem a ter níveis mais altos de colesterol, enquanto pessoas com alto nível de atividade física apresentam níveis mais baixos. Esses padrões são importantes para a construção do modelo preditivo. Observamos que, em geral, há uma tendência de que quanto maior o peso, maior o nível de colesterol. No entanto, também identificamos outliers, ou seja, valores de colesterol muito altos para pesos muito altos. Se encontrássemos casos de pessoas com pesos altos e níveis de colesterol baixos, seria algo estranho. Em seguida, analisamos a relação entre a altura e o nível de colesterol, e não encontramos um padrão claro. Concluímos essa parte da análise e nos preparamos para a próxima etapa, que é cruzar as variáveis numéricas com a variável dependente, o nível de colesterol.Utilizaremos o Seaborn para criar um gráfico chamado pairplot, que mostrará a distribuição das variáveis e suas correlações visuais. Observamos que o peso tem uma correlação positiva com o nível de colesterol, ou seja, quanto maior o peso, maior o nível de colesterol. A altura e a idade estão bem distribuídas ao longo do conjunto de dados. Não encontramos outras correlações significativas entre as variáveis independentes, o que é bom para evitar multicolinearidade em nosso modelo.


#### Análise de Correlação


Nesta aula, aprendemos como converter variáveis categóricas em numéricas para análise de correlação. Utilizamos o método `get_dummies` do Pandas para converter as variáveis nominais, como fumante e grupo sanguíneo, em variáveis binárias. Em seguida, utilizamos o método `factorize` para converter a variável ordinal de nível de atividade física em valores numéricos. Com o DataFrame preparado, plotamos um heatmap de correlação usando o Seaborn, onde observamos as correlações entre as variáveis. Identificamos que o peso tem uma correlação forte com o nível de colesterol, assim como o fato de ser fumante e o tipo sanguíneo. Também notamos uma correlação negativa entre o nível de colesterol e o nível de atividade física.


#### Análise Gráfica com Bucketing


Nesta aula, aprendemos sobre o conceito de bucketing e como aplicá-lo usando a biblioteca Pandas em Python. O bucketing é útil quando queremos analisar ranges de valores em vez de valores individuais. No exemplo dado, criamos buckets para a variável idade, dividindo-a em faixas de 10 anos. Também mostramos como fazer o mesmo para a variável peso. Em seguida, utilizamos gráficos de boxplot para visualizar os agrupamentos e identificar possíveis padrões ou outliers. No caso da idade, não encontramos um padrão claro em relação ao nível de colesterol, ao contrário do que aconteceu com o peso.


### Preparação do Dataset para Treinamento


#### Split do Dataset Treino e Teste


Nesta aula, vamos começar a treinar nosso modelo de predição de colesterol com base em um conjunto de dados. Para isso, vamos utilizar o conceito de pipeline do sklearn, que nos ajuda a automatizar o processo de transformação das variáveis independentes antes de entrar no modelo. Vamos importar os módulos necessários do sklearn, como o modelSelection, linear regression e preprocessing. Também vamos utilizar o column transformer para transformar as colunas de acordo com nossa necessidade. Além disso, vamos tratar os valores ausentes e importar as métricas r2score, minabsoluteerror e minsquarederror. Após preparar o dataframe e fazer o split dos dados de treino e teste, vamos construir o pipeline para treinar nosso modelo.


#### Transformação de Variáveis Categóricas para o Modelo


Nesta aula, vamos aprender sobre a importância de padronizar e codificar variáveis em um pipeline de transformação de dados. Primeiro, discutimos a necessidade de padronizar variáveis numéricas, como idade, altura e peso, para que todas estejam na mesma escala. Em seguida, explicamos a importância de codificar variáveis categóricas nominais usando o método `one-hot encode` e variáveis categóricas ordinais usando o método ordinal encoder. Além disso, mencionamos a importância de imputar valores ausentes usando a mediana para variáveis numéricas. Por fim, mostramos como organizar um pipeline de transformação, criando uma lista de colunas para cada tipo de variável e definindo os passos de transformação para cada categoria.


#### Transformação de Variáveis Ordinais e Numéricas para o Modelo


Neste vídeo, continuamos a nossa pipeline criando um Transformer para as variáveis ordinais. O Transformer irá realizar a mesma função do Imputer, preenchendo os valores ausentes com a moda. A diferença é que precisamos definir a ordem das categorias, que são baixa, moderada e alta. Em seguida, criamos um Transformer numérico, utilizando a mediana como estratégia de preenchimento dos valores ausentes. Em vez de usar um encoder, utilizamos um Scalar, especificamente o Standard Scalar, para converter os valores para a escala Z. Após criar esses três Transformers, encapsulamos todos eles em um Column Transformer, que será o pré-processador antes de treinar o modelo.


### Treinamento do Modelo com Pipelines


Nesta aula, criamos um Pipeline principal para pré-processar os dados e treinar um modelo de regressão linear. O Pipeline é composto por dois steps: o pré-processamento, que transforma as colunas no formato necessário, e o treinamento do modelo. Utilizamos o objeto Pipeline para criar essa estrutura. Em seguida, treinamos o modelo utilizando o método fit, passando o conjunto de treinamento. O Pipeline executa todas as etapas de uma vez, incluindo o pré-processamento e a regressão linear. Podemos visualizar a estrutura do Pipeline e analisar as métricas do modelo treinado.


### Validação do Modelo


#### Análise de Métricas


Nesta aula, vamos analisar as métricas do nosso modelo de predição. Primeiro, vamos gerar a predição usando o conjunto de teste. Em seguida, calcularemos o R2Score, que mede o quão bem as variáveis independentes explicam a variância da variável dependente. Um valor próximo de 1 indica um bom modelo. Também calcularemos o MinAbsoluteError, que mede a diferença média entre os valores preditos e reais, e o RMSE, que leva em conta os outliers. Vamos analisar essas métricas para avaliar a qualidade do nosso modelo.


#### Análise Gráfica de Resíduos


Nesta aula, vamos analisar os resíduos do nosso modelo de regressão. Vamos calcular os resíduos e transformá-los na escala padrão. Em seguida, faremos uma análise gráfica para verificar a linearidade e a homoscedasticidade dos resíduos. Utilizaremos o scatter plot do Seaborn para plotar o gráfico. Observaremos se os valores estão próximos da reta de 45° e se estão dentro da banda de segurança. Além disso, faremos testes estatísticos para confirmar se os resíduos seguem uma distribuição normal. Utilizaremos o gráfico QQ-Plot do módulo Penguin para verificar a normalidade dos resíduos.


#### Teste de Normalidade dos Resíduos


Nesta aula, vamos realizar testes estatísticos para verificar a normalidade dos resíduos. Vamos utilizar os testes de Shapiro-Wilk, Kolmogorov-Smirnov, Lillliefors e Anderson-Darling. Primeiro, importamos os módulos necessários do SciPy e StatsModels. Em seguida, executamos o teste de Shapiro-Wilk, que retorna a estatística do teste e o valor P. Se o valor P for menor que 0.005, podemos rejeitar a hipótese nula de que os resíduos seguem uma distribuição normal. Em seguida, realizamos o teste de Kolmogorov-Smirnov, que também indica a falta de normalidade nos resíduos. Por fim, realizamos o teste de Lillliefors, que indica a normalidade dos resíduos. Para complementar, realizamos o teste de Anderson-Darling, comparando a estatística do teste com os valores críticos para diferentes níveis de significância. Concluímos que três dos quatro testes indicam a falta de normalidade nos resíduos.


#### Teste de Homocedasticidade dos Resíduos


Nesta aula, discutimos sobre o teste estatístico de homoscedasticidade, especificamente o teste de Goldfeld-Quandt. Para realizar esse teste, é necessário preparar os dados, incluindo os resíduos e as variáveis independentes. Como usamos um pipeline de pré-processamento, precisamos criar um novo pipeline apenas com esse passo. Em seguida, aplicamos o pipeline aos dados de teste e executamos o teste de Goldfeld-Quandt. Os resultados mostraram que não podemos rejeitar a hipótese nula, indicando que os resíduos têm variância homogênea em relação às variáveis independentes. Isso é um bom sinal para o modelo. Com isso, concluímos a análise de resíduos e estamos prontos para fazer previsões e entregar o modelo ao usuário final.


### Predição de Valores


Nesta parte final do modelo, vamos aprender como fazer uma predição individual usando o modelo que construímos. Vamos passar os valores desejados para o modelo e ver qual valor ele retorna. É importante lembrar que, no nosso caso, temos uma lista de variáveis para passar. Essas variáveis precisam ser convertidas para que possam entrar no modelo. É aí que entra a beleza dos pipelines. Como já criamos um pipeline, ele já está pronto para receber os dados no seu estado natural e fazer a conversão necessária antes de rodar o modelo e nos trazer a predição. Vamos criar um dicionário com os dados que queremos prever, usando os mesmos nomes de variáveis do nosso dataset. Quando salvamos o modelo, ele salva o pipeline inteiro, não apenas o modelo de regressão linear. Vamos salvar o modelo em um arquivo chamado `model_lr_multiple.plk`. Com isso, concluímos a parte de modelagem e salvamos o modelo. No próximo vídeo, mostraremos como criar uma interface visual simples para o usuário interagir com o modelo.
